In [ ]:
import pandas as pd
import os
from sqlalchemy import create_engine
import logging
import time

logging.basicConfig(
    filename="logs/ingestion_db.log",
    level=logging.DEBUG,
    format="%(asctime)s - %(levelname)s - %(message)s",
    filemode="a"
)   

engine = create_engine('sqlite:///inventory.db')
def ingest_db(df, table_name, engine):
    '''this function will ingest the dataframe into database table'''
    df.to_sql(table_name, con = engine, if_exists = 'replace', index= False)
def load_raw_data():
    '''this function will load the CSVs as dataframe and ingest into db'''
    start = time.time()
    for file in os.listdir('vendor dataset'):
        #print(file)
        if '.csv' in file:
            df = pd.read_csv('vendor dataset/'+file)
            logging.info(f'Ingesting {file} in db')
            ingest_db(df, file[:-4], engine)
    end = time.time()       
    total_time = (end - start)/60
    logging.info('----------------------Ingestion Complete----------------------')
    logging.info(f'\nTotal Time Taken: {total_time} minutes')

if __name__ == '__main__':
    load_raw_data()

In [25]:
# converting .db file to .csv file
import sqlite3
import pandas as pd

# connect to your database
conn = sqlite3.connect("inventory.db")

# list all tables
tables = pd.read_sql("SELECT name FROM sqlite_master WHERE type='table';", conn)
print(tables)

# export each table to CSV
for table in tables['name']:
    df = pd.read_sql(f"SELECT * FROM {table};", conn)
    df.to_csv(f"{table}.csv", index=False)
    print(f"Exported {table}.csv")

conn.close()

                   name
0       begin_inventory
1         end_inventory
2             purchases
3       purchase_prices
4                 sales
5        vendor_invoice
6  vendor_sales_summary
Exported begin_inventory.csv
Exported end_inventory.csv
Exported purchases.csv
Exported purchase_prices.csv
Exported sales.csv
Exported vendor_invoice.csv
Exported vendor_sales_summary.csv
